Dado que vamos a realizar el deploy en Render, debemos minimizar merge que requieran mucha carga de proceso. Por este motivo, voy a generar bases auxiliares para aligerar las funciones PLayTimeGenre y UserForGenre a ser ejecutadas.

In [100]:
import pandas as pd
import pyarrow.parquet as pq
from useful_tools import tipo_de_datos

In [101]:
# Cargar las tablas desde los archivos parquet
steam_games = pq.read_table("steam_games.parquet").to_pandas()
users_items = pq.read_table("users_items.parquet").to_pandas()
user_reviews = pq.read_table("user_reviews.parquet").to_pandas()

Comenzaremos por generar un dataframe con todas las respuestas posibles para PlayTimeGenre

In [102]:
# Convertir 'item_id' a tipo numérico si es necesario
users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

# Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
merged_data_playtimegenre = pd.merge(users_items, steam_games[['id', 'release_date', 'genres']], left_on="item_id", right_on="id", how="left")

# Eliminar la columnas del dataframe resultante
merged_data_playtimegenre.drop(['item_id','item_name','user_id','id', 'items_count', 'playtime_2weeks'], axis=1, inplace=True)

# Eliminar las filas con datos nulos en las columnas "release_date" y "genres"
merged_data_playtimegenre = merged_data_playtimegenre.dropna(subset=['release_date', 'genres'])

# Extraer el año de la columna "release_date"
merged_data_playtimegenre['year'] = merged_data_playtimegenre['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(merged_data_playtimegenre['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])

# Eliminar las filas con valores nulos en la columna "year"
merged_data_playtimegenre.dropna(subset=['year'], inplace=True)

# Eliminar la columna "release_date"
merged_data_playtimegenre.drop('release_date', axis=1, inplace=True)

# Expandir las filas para cada género
merged_data_playtimegenre = merged_data_playtimegenre.explode('genres')

# Eliminar las filas donde el valor en la columna 'genres' es "no genre"
merged_data_playtimegenre = merged_data_playtimegenre[merged_data_playtimegenre['genres'] != "no genre"]

In [103]:
merged_data_playtimegenre

,playtime_forever,genres,year
0,0.001667,Action,2000
1,0.001944,Action,2003
2,1.314722,Action,2010
3,0.514722,Action,2004
4,0.092500,Action,2005
...,...,...,...
3246372,0.000833,Adventure,2015
3246372,0.000833,Free to Play,2015
3246373,0.001111,Casual,2016
3246373,0.001111,Free to Play,2016


In [104]:
# Agrupar por género y año
max_playtime_per_genre = merged_data_playtimegenre.groupby(['genres', 'year'])['playtime_forever'].sum().reset_index()

In [105]:
max_playtime_per_genre

,genres,year,playtime_forever
0,Action,1983,0.964722
1,Action,1984,0.106667
2,Action,1988,4.444722
3,Action,1989,0.168611
4,Action,1990,5.093056
...,...,...,...
344,Web Publishing,2013,92.688333
345,Web Publishing,2014,9.344722
346,Web Publishing,2015,96.853611
347,Web Publishing,2016,0.037778


In [106]:
max_playtime_per_genre.to_parquet('max_playtime_per_genre.parquet')

Tambien generaremos un dataset auxiliar que ya tenga sumados los tiempos de cada usuario agrupado por genero y por año.

In [107]:
# Convertir 'item_id' a tipo numérico si es necesario
users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

# Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
merged_data_userforgenre = pd.merge(users_items, steam_games, left_on="item_id", right_on="id", how="left")

# Eliminar la columnas del dataframe resultante
merged_data_userforgenre.drop(['item_id','price','id','app_name','item_name', 'items_count', 'playtime_2weeks'], axis=1, inplace=True)

# Eliminar las filas con datos nulos en las columnas "release_date" y "genres"
merged_data_userforgenre = merged_data_userforgenre.dropna(subset=['release_date', 'genres'])

# Extraer el año de la columna "release_date"
merged_data_userforgenre['year'] = merged_data_userforgenre['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(merged_data_userforgenre['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])

# Eliminar las filas con valores nulos en la columna "year"
merged_data_userforgenre.dropna(subset=['year'], inplace=True)

# Eliminar la columna "release_date"
merged_data_userforgenre.drop('release_date', axis=1, inplace=True)

# Expandir las filas para cada género
merged_data_userforgenre = merged_data_userforgenre.explode('genres')

# Eliminar las filas donde el valor en la columna 'genres' es "no genre"
merged_data_userforgenre = merged_data_userforgenre[merged_data_userforgenre['genres'] != "no genre"]

merged_data_userforgenre

,user_id,playtime_forever,genres,year
0,76561197970982479,0.001667,Action,2000
1,76561197970982479,0.001944,Action,2003
2,76561197970982479,1.314722,Action,2010
3,76561197970982479,0.514722,Action,2004
4,76561197970982479,0.092500,Action,2005
...,...,...,...,...
3246372,76561198329548331,0.000833,Adventure,2015
3246372,76561198329548331,0.000833,Free to Play,2015
3246373,76561198329548331,0.001111,Casual,2016
3246373,76561198329548331,0.001111,Free to Play,2016


In [108]:
# Agrupar por usuario y calcular la suma de las horas jugadas para cada usuario y genero
user_total_playtime_general = merged_data_userforgenre.groupby(['user_id','genres','year'])['playtime_forever'].sum()
user_total_playtime_general = user_total_playtime_general.reset_index()

user_total_playtime_general

,user_id,genres,year,playtime_forever
0,--000--,Action,2009,1.480278
1,--000--,Action,2010,0.006111
2,--000--,Action,2011,1.811667
3,--000--,Action,2012,30.489444
4,--000--,Action,2013,0.100833
...,...,...,...,...
2915159,zzzmidmiss,Sports,2010,0.053889
2915160,zzzmidmiss,Sports,2014,0.004444
2915161,zzzmidmiss,Strategy,2010,0.002778
2915162,zzzmidmiss,Strategy,2011,0.019167


Ya que el dataset sigue siendo demasiado grande, agruparemos por usuario y genero sumando las horas jugadas y luego seleccionaremos las lineas con mayor tiempo para cada genero dado del dataset original

In [109]:
# Calcular la máxima cantidad de tiempo jugado para cada combinación única de user_id y genres en el dataframe user_total_playtime_general.
max_playtime = user_total_playtime_general.groupby(['user_id','genres'])['playtime_forever'].sum().reset_index()

max_playtime


,user_id,genres,playtime_forever
0,--000--,Action,38.856944
1,--000--,Adventure,3.371667
2,--000--,Casual,4.481944
3,--000--,Early Access,0.147500
4,--000--,Free to Play,5.680000
...,...,...,...
638473,zzzmidmiss,RPG,0.649167
638474,zzzmidmiss,Racing,0.002500
638475,zzzmidmiss,Simulation,0.004444
638476,zzzmidmiss,Sports,0.058333


In [110]:
# Obtener los índices de las filas con las máximas horas jugadas para cada género
indices_max_playtime = max_playtime.groupby('genres')['playtime_forever'].idxmax()

# Seleccionar las filas con esos índices
max_playtime_highest_per_genre = max_playtime.loc[indices_max_playtime]

# Mostrar el DataFrame resultante
max_playtime_highest_per_genre


,user_id,genres,playtime_forever
462692,Sp3ctre,Action,495.070278
449960,REBAS_AS_F-T,Adventure,609.968333
496128,ballig,Animation & Modeling,28.769444
456839,ScottyG555,Animation &amp; Modeling,46.753889
426613,Lickidactyl,Audio Production,30.532222
449961,REBAS_AS_F-T,Casual,342.051111
496131,ballig,Design & Illustration,28.769444
456841,ScottyG555,Design &amp; Illustration,46.753889
17159,76561197978756659,Early Access,88.046944
496134,ballig,Education,28.791944


una vez que tenemos el dataset con los usuarios con mayor tiempo de juego para cada genero, en el dataset original user_total_playtime_general, dejaremos solamente las lineas que pertenezcan a esos usuarios y esos generos.

In [111]:
# Realizar una operación de unión basada en las columnas user_id y genres
user_total_playtime_general = user_total_playtime_general.merge(max_playtime_highest_per_genre[['user_id', 'genres']], on=['user_id', 'genres'], how='inner')

# Mostrar el dataframe filtrado
user_total_playtime_general


,user_id,genres,year,playtime_forever
0,76561197978756659,Early Access,2013,87.996111
1,76561197978756659,Early Access,2015,0.050833
2,76561197995725436,Sports,2012,17.253056
3,76561197995725436,Sports,2013,21.996111
4,76561197995725436,Sports,2014,20.299722
...,...,...,...,...
138,shinomegami,Strategy,2012,44.700556
139,shinomegami,Strategy,2013,40.974444
140,shinomegami,Strategy,2014,35.481944
141,shinomegami,Strategy,2015,0.946389


In [112]:
user_total_playtime_general.to_parquet('user_total_playtime_general.parquet')


generaremos el dataset "top_3_games_per_year" que contendrá todos los juegos con mejores calificaciones de usuarios segun su año de lanzamiento.

In [113]:
# Realizar un left merge entre user_reviews y steam_games
merged_data_usersRecommend = pd.merge(user_reviews[['item_id', 'recommend', 'sentiment_analysis']],
                        steam_games[['id', 'app_name', 'release_date']],
                        left_on='item_id',
                        right_on='id',
                        how='left')
# Eliminando columnas innecesarias y datos nulos
merged_data_usersRecommend.drop(['id', 'item_id'], axis=1, inplace=True)
merged_data_usersRecommend = merged_data_usersRecommend.dropna(subset=['release_date'])

# Extraer el año de la columna "release_date"
merged_data_usersRecommend['year'] = merged_data_usersRecommend['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(merged_data_usersRecommend['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])

# Eliminando columnas innecesarias
merged_data_usersRecommend.drop(['release_date'], axis=1, inplace=True)

# Filtrar los juegos recomendados con sentiment_analysis de 1 o 2
recommended_games = merged_data_usersRecommend[(merged_data_usersRecommend['recommend'] == True) & 
                                (merged_data_usersRecommend['sentiment_analysis'].isin([1, 2]))]

# Eliminando columnas innecesarias
recommended_games.drop(['recommend'], axis=1, inplace=True)

# Agrupar por app_name y sumar los valores de sentiment_analysis
grouped = recommended_games.groupby(['app_name','year'])['sentiment_analysis'].sum().reset_index()

grouped


C:\Users\fabir\AppData\Local\Temp\ipykernel_6904\2600394306.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_games.drop(['recommend'], axis=1, inplace=True)


,app_name,year,sentiment_analysis
0,//N.P.P.D. RUSH//- The milk of Ultraviolet,2014,2
1,0RBITALIS,2015,2
2,"10,000,000",2013,3
3,100% Orange Juice,2013,10
4,100% Orange Juice - Krila & Kae Character Pack,2015,1
...,...,...,...
2636,the static speaks my name,2015,13
2637,theBlu,2012,2
2638,theHunter Classic,2014,44
2639,theHunter: Primal,2015,18


In [114]:
# Definir una función personalizada para seleccionar los tres juegos con el mayor puntaje
def top_n_games(df, n=3):
    return df.nlargest(n, 'sentiment_analysis')

# Aplicar la función a cada grupo por año
top_3_games_per_year = grouped.groupby('year', group_keys=False).apply(top_n_games)


C:\Users\fabir\AppData\Local\Temp\ipykernel_6904\3362378867.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_3_games_per_year = grouped.groupby('year', group_keys=False).apply(top_n_games)


In [115]:
top_3_games_per_year.to_parquet('top_3_games_per_year.parquet')

Generaremos el dataset auxiliar bottom_3_games_per_year que contendrá los 3 peores juegos por año de lanzamiento

In [116]:
# Realizar un left merge entre user_reviews y steam_games
merged_data_usersNotRecommend = pd.merge(user_reviews[['item_id', 'recommend', 'sentiment_analysis']],
                        steam_games[['id', 'app_name', 'release_date']],
                        left_on='item_id',
                        right_on='id',
                        how='left')
# Eliminando columnas innecesarias y datos nulos
merged_data_usersNotRecommend.drop(['id', 'item_id'], axis=1, inplace=True)
merged_data_usersNotRecommend = merged_data_usersNotRecommend.dropna(subset=['release_date'])

# Extraer el año de la columna "release_date"
merged_data_usersNotRecommend['year'] = merged_data_usersNotRecommend['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(merged_data_usersNotRecommend['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])

# Eliminando columnas innecesarias
merged_data_usersNotRecommend.drop(['release_date'], axis=1, inplace=True)

# Filtrar los juegos recomendados con sentiment_analysis de 0
not_recommended_games = merged_data_usersNotRecommend[(merged_data_usersNotRecommend['recommend'] == False) & 
                                (merged_data_usersNotRecommend['sentiment_analysis'].isin([0]))]

# Agrupar por app_name y sumar los valores de sentiment_analysis
groupedNotRecommended = not_recommended_games.groupby(['app_name', 'year']).agg(
    sentiment_analysis=('sentiment_analysis', 'sum'),
    count=('sentiment_analysis', 'count')
).reset_index()

# Eliminando columnas innecesarias
groupedNotRecommended.drop(['sentiment_analysis'], axis=1, inplace=True)

In [117]:
# Definir una función personalizada para seleccionar los tres juegos con el mayor puntaje
def bottom_n_games(df, n=3):
    return df.nlargest(n, 'count')

# Aplicar la función a cada grupo por año
bottom_3_games_per_year = groupedNotRecommended.groupby('year', group_keys=False).apply(bottom_n_games)

C:\Users\fabir\AppData\Local\Temp\ipykernel_6904\941971976.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bottom_3_games_per_year = groupedNotRecommended.groupby('year', group_keys=False).apply(bottom_n_games)


In [118]:
bottom_3_games_per_year.to_parquet('bottom_3_games_per_year.parquet')

In [121]:
# Agregar la columna "release_date" al dataframe "user_reviews" mediante un merge
reviews_with_release = pd.merge(user_reviews, steam_games[['id', 'release_date']], 
                                left_on='item_id', right_on='id', how='left')

# Eliminar todas las filas que tienen datos nulos en la columna "release_date"
reviews_with_release = reviews_with_release.dropna(subset=['release_date'])

# Extraer año de la columna "release_date"
reviews_with_release['year'] = reviews_with_release['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[0].fillna(reviews_with_release['release_date'].str.extract(r'(\d{4})|(\w{3}\s(\d{4}))')[2])

# Eliminando columnas innecesarias
reviews_with_release.drop(['user_id', 'item_id', 'recommend', 'id', 'release_date'], axis=1, inplace=True)

# Contar las reseñas por categoría de sentimiento
sentiment_counts = reviews_with_release.groupby(['sentiment_analysis', 'year']).size().reset_index(name='count')

sentiment_counts_sorted = sentiment_counts

In [122]:
sentiment_counts_sorted.to_parquet('sentiment_counts_sorted.parquet')